# ☁️ Step 3: Training Jobs での機械学習（修正版）

前のノートブックで作成したS3データを使って、**SageMaker Training Jobs**での機械学習を実行します。

## 🎯 このノートブックで学ぶこと
- S3データを使ったTraining Jobsの実行
- クラウド環境でのスケーラブルな機械学習
- Training Jobsの監視と結果取得
- Script Mode vs Training Jobsの違い

## ⏱️ 実行時間: 約8-12分

## 1. 環境設定とS3データパスの確認

In [ ]:
import sagemaker
import boto3
import time
import json
import pandas as pd
import numpy as np
from datetime import datetime
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

# SageMaker設定
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

print("☁️ Training Jobs環境の設定")
print(f"📅 開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📍 Region: {region}")
print(f"👤 Role: {role.split('/')[-1]}")

# S3データパス情報を読み込み
try:
    with open('s3_data_paths.json', 'r') as f:
        s3_paths = json.load(f)
    print("\n✅ S3データパス情報を読み込み完了")
    for data_type, path in s3_paths.items():
        print(f"  📊 {data_type}: {path}")
except FileNotFoundError:
    print("❌ S3パス情報が見つかりません")
    print("🔄 先に 01_data_management.ipynb を実行してください")
    raise

In [ ]:
# 必要なファイルの存在確認
import os

print("🔍 必要なファイルの確認中...")

# scripts/train_lecture.pyの存在確認
script_path = 'scripts/train_lecture.py'
if os.path.exists(script_path):
    print(f"✅ {script_path} が見つかりました")
    
    # ファイルサイズを確認
    file_size = os.path.getsize(script_path)
    print(f"  📄 ファイルサイズ: {file_size:,} bytes")
else:
    print(f"❌ {script_path} が見つかりません")
    print("\n⚠️ エラー: 必要なファイルが見つかりません。")
    print("以下を確認してください:")
    print("1. 正しいフォルダで作業しているか確認")
    print("2. GitHubから正しくクローンされているか確認")
    print("3. scripts/フォルダが存在し、その中にtrain_lecture.pyがあるか確認")
    
    raise FileNotFoundError(f"{script_path} が見つかりません")

# scriptsフォルダの内容を表示
print("\n📁 scriptsフォルダの内容:")
if os.path.exists('scripts'):
    script_files = [f for f in os.listdir('scripts') if f.endswith('.py')]
    for file in sorted(script_files):
        file_path = os.path.join('scripts', file)
        file_size = os.path.getsize(file_path)
        print(f"  📄 {file} ({file_size:,} bytes)")
    
    if not script_files:
        print("  ❌ scriptsフォルダに.pyファイルがありません")
else:
    print("  ❌ scriptsフォルダが存在しません")

## 2. Training Jobs用Estimatorの設定

### 📚 学習ポイント
- SKLearn Estimatorの設定
- インスタンスタイプの選択
- ハイパーパラメータの設定

In [ ]:
print("🔧 Training Jobs用Estimatorを設定中...")

# 講義用Training Jobs設定
lecture_estimator = SKLearn(
    entry_point='scripts/train_lecture.py',  # scriptsフォルダ内のファイルを指定
    framework_version='1.0-1',
    py_version='py3',
    instance_type='ml.m5.large',  # 講義用軽量インスタンス
    instance_count=1,
    role=role,
    hyperparameters={
        'enable-grid-search': 'true',  # グリッドサーチ有効
        'n-jobs': -1  # 並列処理
    },
    base_job_name='lecture-training-jobs',
    max_run=1800  # 最大30分
)

print("✅ Estimator設定完了")
print(f"  💻 インスタンスタイプ: ml.m5.large")
print(f"  🔢 インスタンス数: 1")
print(f"  📄 エントリーポイント: scripts/train_lecture.py")
print(f"  ⚙️ ハイパーパラメータ: グリッドサーチ有効")
print(f"  ⏱️ 最大実行時間: 30分")

# コスト情報
hourly_cost = 0.115  # ml.m5.large の時間単価（USD）
estimated_time_hours = 10 / 60  # 予想実行時間（10分）
estimated_cost = hourly_cost * estimated_time_hours

print(f"\n💰 コスト情報:")
print(f"  💵 時間単価: ${hourly_cost}/時間")
print(f"  ⏱️ 予想実行時間: 約10分")
print(f"  💸 予想コスト: ${estimated_cost:.4f}")

## 3. Training Jobsの実行

### 📚 学習ポイント
- S3データを使ったTraining Jobsの起動
- 実行時間の測定
- ジョブの監視

In [ ]:
print("🚀 Training Jobs実行開始...")
print(f"⏱️ 予想実行時間: 8-12分")
print(f"📊 使用データ: S3に保存された講義用データセット")

training_jobs_start = time.time()

# S3データを使ってTraining Jobsを実行
lecture_estimator.fit({
    'train': s3_paths['train'],
    'validation': s3_paths['validation'],
    'test': s3_paths['test']
})

training_jobs_total_time = time.time() - training_jobs_start

print(f"\n🎉 Training Jobs完了!")
print(f"⏱️ 実際の実行時間: {training_jobs_total_time:.1f}秒 ({training_jobs_total_time/60:.1f}分)")

## 4. Training Jobs結果の詳細分析

### 📚 学習ポイント
- ジョブの実行統計取得
- コスト計算
- ログとアーティファクトの確認

In [ ]:
# Training Jobsの詳細情報取得
training_job_name = lecture_estimator.latest_training_job.job_name
print(f"📋 Training Job詳細情報:")
print(f"  🏷️ ジョブ名: {training_job_name}")

# ジョブの実行統計
training_job_description = sagemaker_session.describe_training_job(training_job_name)

print(f"\n📊 実行統計:")
print(f"  ✅ ジョブ状態: {training_job_description['TrainingJobStatus']}")
print(f"  💻 インスタンスタイプ: {training_job_description['ResourceConfig']['InstanceType']}")
print(f"  🔢 インスタンス数: {training_job_description['ResourceConfig']['InstanceCount']}")

# 実行時間の詳細
if 'TrainingStartTime' in training_job_description and 'TrainingEndTime' in training_job_description:
    start_time = training_job_description['TrainingStartTime']
    end_time = training_job_description['TrainingEndTime']
    actual_training_time = (end_time - start_time).total_seconds()
    print(f"  ⏱️ 実際の訓練時間: {actual_training_time:.1f}秒 ({actual_training_time/60:.1f}分)")

# 課金時間とコスト
billable_seconds = training_job_description.get('BillableTimeInSeconds', 0)
actual_cost = (billable_seconds / 3600) * hourly_cost

print(f"\n💰 コスト詳細:")
print(f"  ⏱️ 課金時間: {billable_seconds}秒 ({billable_seconds/60:.1f}分)")
print(f"  💵 実際のコスト: ${actual_cost:.4f}")
print(f"  📊 予想との差: ${abs(actual_cost - estimated_cost):.4f}")

# モデルアーティファクトとログ
model_artifacts = lecture_estimator.model_data
print(f"\n📦 出力情報:")
print(f"  📄 モデルアーティファクト: {model_artifacts}")
print(f"  📊 CloudWatchログ: /aws/sagemaker/TrainingJobs/{training_job_name}")

# パフォーマンス比較用の情報を保存
training_jobs_info = {
    'job_name': training_job_name,
    'total_time': training_jobs_total_time,
    'training_time': actual_training_time if 'actual_training_time' in locals() else training_jobs_total_time,
    'billable_seconds': billable_seconds,
    'actual_cost': actual_cost,
    'model_artifacts': model_artifacts
}

with open('training_jobs_info.json', 'w') as f:
    json.dump(training_jobs_info, f, indent=2)
    
print(f"\n💾 Training Jobs情報を保存: training_jobs_info.json")

## 5. 推論エンドポイントのテスト（修正版）

### 📚 学習ポイント
- Training Jobsで訓練したモデルのデプロイ
- エンドポイントでの推論テスト
- エンドポイントのコスト管理
- エラーハンドリング

In [ ]:
print("🔮 推論エンドポイントのテスト開始...")
endpoint_start = time.time()

# エンドポイント作成
print("  📡 エンドポイントを作成中...")
lecture_predictor = lecture_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',  # 推論用軽量インスタンス
    endpoint_name=f'lecture-endpoint-{int(time.time())}'
)

endpoint_creation_time = time.time() - endpoint_start
print(f"  ✅ エンドポイント作成完了: {endpoint_creation_time:.1f}秒")
print(f"  🏷️ エンドポイント名: {lecture_predictor.endpoint_name}")

# テストデータで推論（修正版）
print(f"\n🎯 推論テスト実行中...")

# S3からテストデータを読み込み（少量）
test_data_sample = pd.read_csv(s3_paths['test']).head(5)

# データの前処理（訓練時と同じ形式に変換）
print(f"  📊 テストデータ形状: {test_data_sample.shape}")
print(f"  📋 カラム数: {len(test_data_sample.columns)}")

# targetカラムを除外してnumpy配列に変換
if 'target' in test_data_sample.columns:
    test_samples = test_data_sample.drop('target', axis=1).values
    actual_labels = test_data_sample['target'].values
else:
    test_samples = test_data_sample.values
    actual_labels = None
    print("  ⚠️ targetカラムが見つかりません")

print(f"  🎯 推論用データ形状: {test_samples.shape}")

# データ型を確認・修正
test_samples = test_samples.astype(np.float32)
print(f"  📊 データ型: {test_samples.dtype}")

try:
    inference_start = time.time()
    
    # 推論実行（エラーハンドリング付き）
    print("  🔮 推論を実行中...")
    predictions = lecture_predictor.predict(test_samples)
    
    inference_time = time.time() - inference_start
    
    print(f"  ✅ 推論成功!")
    print(f"  ⏱️ 推論時間: {inference_time:.4f}秒 (5サンプル)")
    print(f"  🚀 平均推論時間: {inference_time/5:.4f}秒/サンプル")
    
    # 結果の表示
    if actual_labels is not None:
        print(f"\n📊 推論結果:")
        correct_predictions = 0
        for i, (pred, actual) in enumerate(zip(predictions, actual_labels)):
            status = "✅" if pred == actual else "❌"
            if pred == actual:
                correct_predictions += 1
            print(f"  サンプル {i+1}: 予測={pred}, 実際={actual} {status}")
        
        accuracy = correct_predictions / len(predictions)
        print(f"\n🎯 推論精度（5サンプル）: {accuracy:.2f} ({correct_predictions}/{len(predictions)})")
    else:
        print(f"\n📊 推論結果: {predictions}")

except Exception as e:
    print(f"\n❌ 推論エラーが発生しました: {str(e)}")
    print("\n🔍 トラブルシューティング:")
    print("1. エンドポイントが正常に起動しているか確認")
    print("2. データ形式が訓練時と一致しているか確認")
    print("3. CloudWatchログを確認してください")
    print(f"   ログURL: https://console.aws.amazon.com/cloudwatch/home?region={region}#logEventViewer:group=/aws/sagemaker/Endpoints/{lecture_predictor.endpoint_name}")
    
    # エラーでも処理を継続
    print("\n⚠️ 推論テストはスキップして続行します")
    inference_time = 0
    accuracy = 0

# エンドポイントのコスト情報
endpoint_hourly_cost = 0.056  # ml.t2.medium の時間単価
print(f"\n💰 エンドポイントコスト情報:")
print(f"  💻 インスタンス: ml.t2.medium")
print(f"  💵 時間単価: ${endpoint_hourly_cost}/時間")
print(f"  📅 月額概算: ${endpoint_hourly_cost * 24 * 30:.2f} (24時間稼働の場合)")
print(f"  ⚠️ 重要: 使用後は必ずエンドポイントを削除してください")

## 6. リソースのクリーンアップ

In [ ]:
# エンドポイントの削除（重要！）
print("🗑️ エンドポイントを削除中...")
try:
    lecture_predictor.delete_endpoint()
    print("✅ エンドポイント削除完了")
except Exception as e:
    print(f"⚠️ エンドポイント削除エラー: {str(e)}")
    print("💡 手動でAWSコンソールから削除してください")

print(f"\n🎉 Training Jobs学習完了!")
print(f"📊 実行結果サマリー:")
print(f"  ⏱️ Training Jobs実行時間: {training_jobs_total_time:.1f}秒")
print(f"  💰 Training Jobsコスト: ${actual_cost:.4f}")
print(f"  🔮 推論テスト: {'成功' if 'accuracy' in locals() and accuracy > 0 else 'スキップ'}")
print(f"  🗑️ リソースクリーンアップ: 完了")

print(f"\n🎯 Training Jobsの特徴を体験しました:")
features = [
    "S3データを使った自動的なデータ取得",
    "スケーラブルなクラウド環境での訓練",
    "自動的なモデルアーティファクト保存",
    "推論エンドポイントへの簡単デプロイ",
    "詳細な実行統計とコスト管理",
    "エラーハンドリングとトラブルシューティング"
]

for i, feature in enumerate(features, 1):
    print(f"  {i}. ✅ {feature}")

print(f"\n🚀 次のノートブック: 04_comparison_and_summary.ipynb")
print(f"   Script ModeとTraining Jobsの詳細比較を行います！")